In [1]:
import numpy as np
import pandas as pd
#from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
#from IPython.core.debugger import Tracer
#import scipy as sp

from pylab import rcParams
rcParams['figure.figsize'] = 5, 8

%matplotlib inline

# Exercise One

In [66]:
y = np.array([2,3,0,-1])
x = np.array([4,-2,1,
              5,-3,2,
              1,1,3,
              0,1,4]).reshape(4,3)
lam = 1
beta = np.zeros(3)

In [67]:
x

array([[ 4, -2,  1],
       [ 5, -3,  2],
       [ 1,  1,  3],
       [ 0,  1,  4]])

In [68]:
x[:,0:3]

array([[ 4, -2,  1],
       [ 5, -3,  2],
       [ 1,  1,  3],
       [ 0,  1,  4]])

In [69]:
x[:, (0,2)]

array([[4, 1],
       [5, 2],
       [1, 3],
       [0, 4]])

In [70]:
x[:, (0,1,2)]

array([[ 4, -2,  1],
       [ 5, -3,  2],
       [ 1,  1,  3],
       [ 0,  1,  4]])

In [71]:
x[:, [0,2]]

array([[4, 1],
       [5, 2],
       [1, 3],
       [0, 4]])

In [72]:
x[:, np.arange(x.shape[1])]

array([[ 4, -2,  1],
       [ 5, -3,  2],
       [ 1,  1,  3],
       [ 0,  1,  4]])

In [73]:
x.shape[1]

3

In [74]:
idxs = list(range(x.shape[1]))
idxs.remove(0)
x[:, idxs]

array([[-2,  1],
       [-3,  2],
       [ 1,  3],
       [ 1,  4]])

In [75]:
x[:, foo]

array([[4, 1],
       [5, 2],
       [1, 3],
       [0, 4]])

In [76]:
beta

array([ 0.,  0.,  0.])

In [77]:
beta[idxs]

array([ 0.,  0.])

In [78]:
np.arange(3)[idxs]

array([1, 2])

In [79]:
foox = x[:, (1,2)]
foox

array([[-2,  1],
       [-3,  2],
       [ 1,  3],
       [ 1,  4]])

In [80]:
foobeta = beta[[1,2]]
foobeta

array([ 0.,  0.])

In [81]:
foox.dot(foobeta)

array([ 0.,  0.,  0.,  0.])

In [82]:
y

array([ 2,  3,  0, -1])

In [83]:
y - foox.dot(foobeta)

array([ 2.,  3.,  0., -1.])

In [85]:
x[:,0]

array([4, 5, 1, 0])

In [86]:
x[:,0] * y - foox.dot(foobeta)

array([  8.,  15.,   0.,   0.])

In [87]:
sum(x[:,0] * y - foox.dot(foobeta))

23.0

In [88]:
2 * sum(x[:,0] * y - foox.dot(foobeta))

46.0

In [89]:
.536*4 + 0*1

2.144

In [90]:
.536*5

2.68

In [91]:
.536*1

0.536

In [92]:
3-2.68

0.31999999999999984

In [93]:
-2*-.144

0.288

In [94]:
-3*.32

-0.96

In [95]:
.288-0.96-0.536-1

-2.208

In [96]:
-2.208*2

-4.416

In [98]:
foo_x_minus = x[:, [0,2]]
foo_x_minus

array([[4, 1],
       [5, 2],
       [1, 3],
       [0, 4]])

In [99]:
foo_beta_minus = np.array([0.536, 0])
foo_beta_minus

array([ 0.536,  0.   ])

In [101]:
2 * sum(x[:, 0] * y - foo_x_minus.dot(foo_beta_minus))

35.280000000000001

In [102]:
foo_x_minus.dot(foo_beta_minus)

array([ 2.144,  2.68 ,  0.536,  0.   ])

In [104]:
y - foo_x_minus.dot(foo_beta_minus)

array([-0.144,  0.32 , -0.536, -1.   ])

In [105]:
2 * sum(x[:, 1] * y - foo_x_minus.dot(foo_beta_minus))

-38.719999999999999

In [107]:
y - foo_x_minus.dot(foo_beta_minus)

array([-0.144,  0.32 , -0.536, -1.   ])

In [109]:
x[:, 1]

array([-2, -3,  1,  1])

In [112]:
x[:, 1] * (y - foo_x_minus.dot(foo_beta_minus))

array([ 0.288, -0.96 , -0.536, -1.   ])

In [113]:
2 * sum(x[:, 1] * (y - foo_x_minus.dot(foo_beta_minus)))

-4.4159999999999986

In [115]:
x[:, 0]

array([4, 5, 1, 0])

In [116]:
x[:, 0]**2

array([16, 25,  1,  0])

In [117]:
2 * sum(x[:, 1]**2)

30

In [120]:
(-4.4159999999999986 + 1) / 30

-0.11386666666666662

In [191]:
from coorddescent import *

In [192]:
get_sequence_of_js(3,3)

NameError: name 'get_sequence_of_js' is not defined

In [130]:
c_term(beta, x, y, 3, 10)

8.0

In [131]:
minimize_beta_term(beta, x, y, 1, lam)

0.5357142857142857

In [137]:
minimize_beta_term(np.array([0.5357142857142857, 0, 0]), x, y, 2, lam)

-0.11428571428571432

In [138]:
minimize_beta_term(np.array([0.5357142857142857, 
                             -0.11428571428571432, 0]), x, y, 3, lam)

-0.15738095238095237

In [194]:
minimize_beta_term(np.array([0.5357142857142857, 
                             -0.11428571428571432, 
                             -0.15738095238095237]), x, y, 1, lam)

0.53955215419501124

In [195]:
minimize_beta_term(np.array([0.53955215419501124, 
                             -0.11428571428571432, 
                             -0.15738095238095237]), x, y, 2, lam)

-0.11914890400604702

In [196]:
minimize_beta_term(np.array([0.53955215419501124, 
                             -0.11914890400604702, 
                             -0.15738095238095237]), x, y, 3, lam)

-0.15971785084404125

In [139]:
x.shape[1]

3

In [144]:
np.repeat?

In [167]:
np.repeat(y, 2, 0)

array([ 2,  2,  3,  3,  0,  0, -1, -1])

In [148]:
y

array([ 2,  3,  0, -1])

In [149]:
np.array(y)

array([ 2,  3,  0, -1])

In [150]:
np.ravel(y)

array([ 2,  3,  0, -1])

In [155]:
y[np.newaxis]

array([[ 2,  3,  0, -1]])

In [170]:
np.repeat(y[np.newaxis], 2, 0).flatten()

array([ 2,  3,  0, -1,  2,  3,  0, -1])

In [158]:
y[np.newaxis]

array([[ 2,  3,  0, -1]])

In [171]:
np.repeat(np.arange(3)[np.newaxis], 2, 0).flatten()

array([0, 1, 2, 0, 1, 2])

In [172]:
np.repeat(np.arange(x.shape[1])[np.newaxis], 2, 0).flatten()

array([0, 1, 2, 0, 1, 2])

In [173]:
np.repeat(np.arange(x.shape[1])[np.newaxis], 10, 0).flatten()

array([0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1,
       2, 0, 1, 2, 0, 1, 2])

In [174]:
max_iter = 10

In [175]:
max_iter / x.shape[1]

3.3333333333333335

In [177]:
np.repeat(np.arange(x.shape[1])[np.newaxis], 3, 0).flatten()

array([0, 1, 2, 0, 1, 2, 0, 1, 2])

In [179]:
for i in range(3):
    print(i)

0
1
2


In [182]:
np.arange(1,4)

array([1, 2, 3])

In [183]:
?np.repeat

In [186]:
np.append(np.arange(1,4), np.arange(1,4))

array([1, 2, 3, 1, 2, 3])